In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from pandas_profiling import ProfileReport
import nltk
from nltk.corpus import stopwords
from spacy.lang.tr import Turkish
from sklearn.preprocessing import LabelEncoder
import kaleido

In [170]:
tweets_d = pd.read_csv("../data/depresyon_1000_df.csv", parse_dates=True)
tweets_m=pd.read_csv("../data/mutluluk_1000_df.csv", parse_dates=True)

In [171]:
tweets_d["text"]=tweets_d["text"].str.replace(r'depresyon', '', regex=True)
tweets_d["label"] = "depresyon"
tweets_m["text"]=tweets_m["text"].str.replace(r'mutluluk', '', regex=True)
tweets_m["label"] = "mutluluk"

In [205]:
tweets = pd.concat([tweets_d, tweets_m])
tweets.drop(columns="Unnamed: 0", inplace=True)
tweets['datetime'] = pd.to_datetime(tweets["datetime"], utc=True)
df = tweets.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2002 entries, 0 to 1000
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   datetime          2002 non-null   datetime64[ns, UTC]
 1   tweet_id          2002 non-null   int64              
 2   text              2002 non-null   object             
 3   username          2002 non-null   object             
 4   quoted_tweet      253 non-null    object             
 5   retweeted_tweet   0 non-null      float64            
 6   in_reply_to_user  0 non-null      float64            
 7   tweet_lang        2002 non-null   object             
 8   label             2002 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(2), int64(1), object(5)
memory usage: 156.4+ KB


In [173]:
df.tail()

,datetime,tweet_id,text,username,quoted_tweet,retweeted_tweet,in_reply_to_user,tweet_lang,label
996,2022-08-08 12:37:35+00:00,1556620638160314377,Kısmet ve Hasan kardeşlerimizin nikah törenler...,uner_mesut,NaN,NaN,NaN,tr,mutluluk
997,2022-08-08 12:37:18+00:00,1556620566458765313,Mutluluk bile haddini aşınca acıya dönüşür.. 🕸️🍷,aleyna_sedal,NaN,NaN,NaN,tr,mutluluk
998,2022-08-08 12:37:15+00:00,1556620555792564224,Hayattan izlenimler: Genelde insanlar artık HA...,bulutsuzluk54,NaN,NaN,NaN,tr,mutluluk
999,2022-08-08 12:37:06+00:00,1556620516663955456,"""Mutluluk bir kelebek gibidir, kovalarsan hep ...",iremmarcc,NaN,NaN,NaN,tr,mutluluk
1000,2022-08-08 12:36:27+00:00,1556620353354473472,Ne kadar çooook gereksiz insan biriktirmişiz h...,heso_06,NaN,NaN,NaN,tr,mutluluk


In [174]:
df.label.value_counts()

depresyon    1001
mutluluk     1001
Name: label, dtype: int64

In [359]:
group_df=df.groupby([df.datetime.dt.hour,'label'], as_index=True).agg("count")
group_df.head(50)

,,datetime,tweet_id,text,username,quoted_tweet,retweeted_tweet,in_reply_to_user,tweet_lang
datetime,label,,,,,,,,
0,depresyon,16,16,16,16,3,0,0,16
1,depresyon,5,5,5,5,0,0,0,5
2,depresyon,6,6,6,6,0,0,0,6
3,depresyon,4,4,4,4,0,0,0,4
4,depresyon,5,5,5,5,0,0,0,5
5,depresyon,12,12,12,12,3,0,0,12
6,depresyon,14,14,14,14,3,0,0,14
7,depresyon,23,23,23,23,2,0,0,23
8,depresyon,33,33,33,33,2,0,0,33


In [339]:
pivot_t=pd.pivot_table(df, values="text", columns=["label"], aggfunc="count", index=df.datetime.dt.hour)

In [349]:
pivot_t

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23],
           dtype='int64', name='datetime')

In [377]:
fig = px.bar(data_frame=pivot_t, x=pivot_t.index,
       y=["mutluluk","depresyon"],
       labels={
    "datetime": "Hour",
    "value": "Count"
                 })
fig.show()

In [175]:
profile = ProfileReport(df, title="Pandas Profiling Report")

In [176]:
# profile.to_file("tweets_repot.html")

In [177]:
df["text"].str.lower()

0                 günlerce yataktan çıkmamalı  loading...
1       aylardır büyüdüm mü yoksa ağır bir  içerisinde...
2                                       depresyon güzelim
3       merve özbey-helal ettim, yıllarımın  şarkısıdı...
4       şu taş yığınlarından farkımız olsun artik # 🙇‍...
                              ...                        
996     kısmet ve hasan kardeşlerimizin nikah törenler...
997      mutluluk bile haddini aşınca acıya dönüşür.. 🕸️🍷
998     hayattan izlenimler: genelde insanlar artık ha...
999     "mutluluk bir kelebek gibidir, kovalarsan hep ...
1000    ne kadar çooook gereksiz insan biriktirmişiz h...
Name: text, Length: 2002, dtype: object

In [178]:
df['text'] = df['text'].str.replace(r'[^\w\s]+', '')

C:\Users\ahmet\AppData\Local\Temp\ipykernel_10868\2913396941.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.



In [179]:
df.text.head()

0                 Günlerce yataktan çıkmamalı  loading
1    Aylardır büyüdüm mü yoksa ağır bir  içerisinde...
2                                    Depresyon güzelim
3    Merve özbeyhelal ettim yıllarımın  şarkısıdır ...
4    Şu taş yığınlarından farkımız olsun artik   ht...
Name: text, dtype: object

In [180]:
df.text=df["text"].str.replace(r'\d', '', regex=True)

df.head()

,datetime,tweet_id,text,username,quoted_tweet,retweeted_tweet,in_reply_to_user,tweet_lang,label
0,2022-08-08 23:53:20+00:00,1556790696450920451,Günlerce yataktan çıkmamalı loading,flamingoturnasi,NaN,NaN,NaN,tr,depresyon
1,2022-08-08 23:46:09+00:00,1556788889372721158,Aylardır büyüdüm mü yoksa ağır bir içerisinde...,yananbirgenclik,NaN,NaN,NaN,tr,depresyon
2,2022-08-08 23:22:36+00:00,1556782962133401604,Depresyon güzelim,cikolatasizkal,NaN,NaN,NaN,tr,depresyon
3,2022-08-08 23:18:12+00:00,1556781853771808768,Merve özbeyhelal ettim yıllarımın şarkısıdır ...,thebetebet,https://twitter.com/archivery/status/155670388...,NaN,NaN,tr,depresyon
4,2022-08-08 23:17:32+00:00,1556781685655699456,Şu taş yığınlarından farkımız olsun artik ht...,ZehraaF,NaN,NaN,NaN,tr,depresyon


In [181]:
nltk.download("stopwords")

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [182]:
sw = stopwords.words("Turkish")
print(sw)

['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


In [183]:
df.text = df["text"].apply(lambda words: ' '.join(word.lower() for word in str(words).split() if word not in sw))
df.text.head()

0                  günlerce yataktan çıkmamalı loading
1    aylardır büyüdüm yoksa ağır bir içerisinde miy...
2                                    depresyon güzelim
3    merve özbeyhelal ettim yıllarımın şarkısıdır o...
4    şu taş yığınlarından farkımız olsun artik http...
Name: text, dtype: object

In [184]:
all_ = [x for y in df["text"] for x in y.split(' ') ]
a,b = np.unique(all_, return_counts = True)
to_remove = a[b<3]
df["text"] = [' '.join(np.array(y.split(' '))[~np.isin(y.split(' '), to_remove)]) for y in df["text"]]
df.text.head()

0                                             günlerce
1    aylardır büyüdüm yoksa ağır bir içerisinde miy...
2                                    depresyon güzelim
3                                                ettim
4                                         şu taş olsun
Name: text, dtype: object

In [185]:
encoder = LabelEncoder()
df["label"] = encoder.fit_transform(df.label)

In [186]:
df["label"].value_counts()

0    1001
1    1001
Name: label, dtype: int64